# Análise dos Ritmos de contágio municipais para o G1 

Para uma melhor compreensão da situação dos municípios no combate a Pandemia da Covid-19, analisamos 5% das cidades com mais mortes para a verificação dos ritmos de contágio e análise desses municípios.


*Code made by: Francisco Nogueira*

## Importing Standard Libraries

In [1]:
%reload_ext autoreload
%autoreload 2


import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import warnings

warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Centering and fixing title
def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))


In [2]:
from aux import *
import matplotlib.pyplot as plt

## Load Data

In [3]:
df = pd.read_csv('http://datasource.coronacidades.org:7000/br/cities/farolcovid/main')
df = df.sort_values('city_name')\
                    .reset_index(drop=True)

df.head()

,city_id,city_name,state_id,state_name,population,health_system_region,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_ventilators,author_number_ventilators,number_ventilators,confirmed_cases,active_cases,deaths,notification_rate,state_notification_rate,last_updated_subnotification,subnotification_place_type,subnotification_rate,subnotification_rank,subnotification_classification,last_updated_rt,rt_10days_ago_low,rt_10days_ago_high,rt_10days_ago_most_likely,rt_17days_ago_low,rt_17days_ago_high,rt_17days_ago_most_likely,rt_classification,rt_10days_ago_avg,rt_17days_ago_avg,rt_ratio_week_avg,rt_growth,last_updated_inloco,inloco_today_7days_avg,inloco_last_week_7days_avg,inloco_ratio_week_avgs,inloco_growth,dday_beds_best,dday_beds_worst,dday_classification,dday_beds_best_months,overall_alert,data_last_refreshed
0,5200050,Abadia de Goiás,GO,Goiás,8773,Central,2020-04-01,DataSUS,0,2020-04-01,DataSUS,0.0,9.0,5.0,0.0,0.635980,0.635980,2020-06-09 16:55:47,state,0.364020,NaN,NaN,2020-06-09 17:34:37,0.56,2.75,1.58,0.64,4.81,2.35,NaN,1.778571,NaN,NaN,NaN,2020-06-09 16:57:04,0.342280,0.360209,0.950225,estabilizando,NaN,NaN,NaN,NaN,NaN,2020-06-09 17:35:32
1,3100104,Abadia dos Dourados,MG,Minas Gerais,6989,Uberlândia,2020-04-01,DataSUS,11,2020-04-01,DataSUS,0.0,5.0,0.0,0.0,0.811237,0.811237,2020-06-09 16:55:47,state,0.188763,NaN,NaN,2020-06-09 17:34:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-09 16:57:04,0.420234,0.409073,1.027284,estabilizando,NaN,NaN,NaN,NaN,NaN,2020-06-09 17:35:32
2,5200100,Abadiânia,GO,Goiás,20042,Pirineus,2020-04-01,DataSUS,9,2020-04-01,DataSUS,0.0,12.0,19.0,0.0,0.635980,0.635980,2020-06-09 16:55:47,state,0.364020,NaN,NaN,2020-06-09 17:34:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-09 16:57:04,0.386274,0.382408,1.010109,estabilizando,NaN,NaN,NaN,NaN,NaN,2020-06-09 17:35:32
3,1500107,Abaetetuba,PA,Pará,157698,Alça Viária,2020-04-01,DataSUS,214,2020-04-01,DataSUS,10.0,1735.0,1783.0,68.0,0.527846,0.289335,2020-06-09 16:55:47,city,0.472154,46.0,bom,2020-06-09 17:34:37,1.76,1.99,1.89,1.50,1.83,1.67,ruim,1.972857,1.634286,1.207168,piorando,2020-06-09 16:57:04,0.447712,0.484150,0.924738,estabilizando,2.0,2.0,ruim,1.0,alto,2020-06-09 17:35:32
4,3100203,Abaeté,MG,Minas Gerais,23237,Sete Lagoas,2020-04-01,DataSUS,32,2020-04-01,DataSUS,2.0,2.0,0.0,0.0,0.811237,0.811237,2020-06-09 16:55:47,state,0.188763,NaN,NaN,2020-06-09 17:34:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-09 16:57:04,0.356360,0.366233,0.973044,estabilizando,NaN,NaN,NaN,NaN,NaN,2020-06-09 17:35:32


In [4]:
# Indicador sobre atualização dos dados e total de mortos
print("Total de mortos: ", df['deaths'].sum())

print("Atualizado em: ", df['data_last_refreshed'].max())

print("Rt atualizado em: ", df['last_updated_subnotification'].min())

Total de mortos:  36856.0
Atualizado em:  2020-06-09 17:35:32
Rt atualizado em:  2020-06-09 16:55:47


In [5]:
dfstates = pd.read_csv('http://datasource.coronacidades.org:7000/br/states/farolcovid/main')

Distribuição do total de mortos popr municípios

In [6]:
#Filtrando dados que apresentam cálculo de Rt
dfna = df.loc[df['rt_ratio_week_avg'].isna()]
dfnna = df.loc[~df['rt_ratio_week_avg'].isna()]

In [7]:
# 95° Percentil
p95_deaths = np.percentile(dfnna['deaths'],95)
print(f"95° Percentil de Mortos: {p95_deaths:.2f}")

95° Percentil de Mortos: 35.70


In [8]:
#Filtra apenas os municípios que concentram mais mortes
dfnfilt = dfnna.query(f"deaths >= {p95_deaths}")\
            .sort_values(['rt_ratio_week_avg'])\
            .reset_index(drop=True)

print("Total de Municípios analisado: %d" % dfnfilt.shape[0])

Total de Municípios analisado: 124


In [9]:
dfnfilt

,city_id,city_name,state_id,state_name,population,health_system_region,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_ventilators,author_number_ventilators,number_ventilators,confirmed_cases,active_cases,deaths,notification_rate,state_notification_rate,last_updated_subnotification,subnotification_place_type,subnotification_rate,subnotification_rank,subnotification_classification,last_updated_rt,rt_10days_ago_low,rt_10days_ago_high,rt_10days_ago_most_likely,rt_17days_ago_low,rt_17days_ago_high,rt_17days_ago_most_likely,rt_classification,rt_10days_ago_avg,rt_17days_ago_avg,rt_ratio_week_avg,rt_growth,last_updated_inloco,inloco_today_7days_avg,inloco_last_week_7days_avg,inloco_ratio_week_avgs,inloco_growth,dday_beds_best,dday_beds_worst,dday_classification,dday_beds_best_months,overall_alert,data_last_refreshed
0,1501808,Breves,PA,Pará,102701,Marajó Ocidental,2020-04-01,DataSUS,88,2020-04-01,DataSUS,26.0,634.0,920.0,63.0,0.189072,0.289335,2020-06-09 16:55:47,city,0.810928,116.0,ruim,2020-06-09 17:34:37,0.96,1.16,1.08,1.26,1.46,1.37,insatisfatório,1.192857,1.548571,0.770295,melhorando,2020-06-09 16:57:04,0.445415,0.490498,0.908087,estabilizando,1.0,1.0,ruim,1.0,alto,2020-06-09 17:35:32
1,1504422,Marituba,PA,Pará,131521,Metropolitana,2020-04-01,DataSUS,106,2020-04-01,DataSUS,20.0,497.0,823.0,48.0,0.206499,0.289335,2020-06-09 16:55:47,city,0.793501,108.0,ruim,2020-06-09 17:34:37,1.48,1.73,1.62,1.92,2.27,2.11,ruim,1.808571,2.304286,0.784873,melhorando,2020-06-09 16:57:04,0.381179,0.406587,0.937510,estabilizando,9.0,8.0,ruim,1.0,alto,2020-06-09 17:35:32
2,2306405,Itapipoca,CE,Ceará,129358,Itapipoca,2020-04-01,DataSUS,85,2020-04-01,DataSUS,20.0,1158.0,1260.0,63.0,0.365033,0.306610,2020-06-09 16:55:47,city,0.634967,78.0,insatisfatório,2020-06-09 17:34:37,1.27,1.48,1.38,1.58,1.83,1.71,ruim,1.498571,1.888571,0.793495,melhorando,2020-06-09 16:57:04,0.473387,0.512166,0.924284,estabilizando,1.0,1.0,ruim,1.0,alto,2020-06-09 17:35:32
3,2309706,Pacatuba,CE,Ceará,83432,Maracanaú,2020-04-01,DataSUS,16,2020-04-01,DataSUS,0.0,673.0,1081.0,43.0,0.323887,0.306610,2020-06-09 16:55:47,city,0.676113,89.0,insatisfatório,2020-06-09 17:34:37,1.26,1.54,1.42,1.47,1.80,1.65,ruim,1.474286,1.832857,0.804365,melhorando,2020-06-09 16:57:04,0.435390,0.460680,0.945102,estabilizando,1.0,1.0,ruim,1.0,alto,2020-06-09 17:35:32
4,1304203,Tefé,AM,Amazonas,59849,Triângulo,2020-04-01,DataSUS,35,2020-04-01,DataSUS,2.0,2269.0,1333.0,63.0,0.710481,0.428153,2020-06-09 16:55:47,city,0.289519,28.0,bom,2020-06-09 17:34:37,1.21,1.40,1.32,1.51,1.73,1.64,ruim,1.440000,1.784286,0.807046,melhorando,2020-06-09 16:57:04,0.411505,0.457777,0.898919,piorando,1.0,1.0,ruim,1.0,alto,2020-06-09 17:35:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,3551009,São Vicente,SP,São Paulo,365798,Baixada Santista,2020-04-01,DataSUS,191,2020-04-01,DataSUS,67.0,1352.0,2519.0,91.0,0.280630,0.305145,2020-06-09 16:55:47,city,0.719370,161.0,ruim,2020-06-09 17:34:37,1.75,2.00,1.89,1.53,1.88,1.72,ruim,1.837143,1.642857,1.118261,piorando,2020-06-09 16:57:04,0.416872,0.420543,0.991271,estabilizando,1.0,1.0,ruim,1.0,alto,2020-06-09 17:35:32
120,2504009,Campina Grande,PB,Paraíba,409731,III Núcleo Regional de Saúde,2020-04-01,DataSUS,1087,2020-04-01,DataSUS,170.0,2650.0,1780.0,44.0,1.000000,0.814311,2020-06-09 16:55:47,city,0.000000,6.0,bom,2020-06-09 17:34:37,1.64,1.95,1.80,1.52,1.96,1.75,ruim,1.834286,1.592857,1.151570,piorando,2020-06-09 16:57:04,0.431114,0.459431,0.938365,estabilizando,46.0,34.0,insatisfatório,2.0,alto,2020-06-09 17:35:32
121,1501501,Benevides,PA,Pará,62737,Metropolitana,2020-04-01,DataSUS,0,2020-04-01,DataSUS,0.0,1034.0,1370.0,43.0,0.454830,0.289335,2020-06-09 16:55:47,city,0.545170,53.0,insatisfatório,2020-06-09 17:34:37,1.55,1.83,1.70,1.74,2.14,1.95,ruim,1.887143,1.622857,1.162852,piorando,2020-06-09 16:57:04,0.375134,0.411

In [10]:
#Municípios com mais de 36 mortes por estados
x = list(set(dfnfilt['state_id']))
dict(zip(x,[sum(dfnfilt['state_id']==i) for i in x]))

{'PE': 13,
 'PB': 3,
 'AP': 2,
 'RO': 1,
 'AC': 1,
 'GO': 1,
 'CE': 7,
 'PR': 1,
 'SE': 1,
 'BA': 2,
 'PA': 17,
 'AM': 7,
 'PI': 1,
 'ES': 4,
 'DF': 1,
 'AL': 1,
 'RJ': 19,
 'RN': 2,
 'RS': 1,
 'MA': 4,
 'SP': 32,
 'RR': 1,
 'MG': 2}

In [11]:
#Calculo de mortes/100k Hab.
def deaths_per_100khab(df, col_deaths="deaths",col_pop='population'):
    return 100e3 * df[col_deaths]/df[col_pop]

dfnfilt['deaths_100khab'] = deaths_per_100khab(dfnfilt)
dfnfilt['cases_100khab'] = deaths_per_100khab(dfnfilt, col_deaths="confirmed_cases")

In [12]:
#Salvar os arquivos em .csv
dfnfilt[['city_name','deaths','deaths_100khab',"inloco_today_7days_avg","inloco_last_week_7days_avg", "inloco_ratio_week_avgs",'rt_17days_ago_avg','rt_10days_ago_avg','rt_ratio_week_avg']].to_csv("data/output/dfnfilt.csv", index=False)

In [13]:
#DataFrame com os dados ordenados
##Alterando o nome da coluna, altera a ordenação dos dados, as 10 primeiras linhas serão
## as 10 menores observações, as 10 ultimas, serão as 10 maiores
### e.g. Municipios com o maior Rt, municípios com o menor Rt ...
dfexib = dfnfilt.sort_values('rt_10days_ago_avg') #Alterar essa linha

### Visualização da relação do Rt com outras variáveis

In [14]:
# Gráfico de Dispersão entre variáveis
## Usar para análise exploratória
x = 'deaths_100khab' #Alterar essa linha
y = 'rt_ratio_week_avg' #Alterar essa linha
z = dfexib.sort_values(x)

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=list(z[x]), y=list(z[y]),
               mode='markers',text=z['city_name']))


fig.layout.xaxis.update({'title': x})
fig.layout.yaxis.update({'title': 'rt_ratio_week_avg'})
fig.update_layout(title= '$$ΔR_t X Mortes/100k hab$$')
fig.show()

In [15]:
# Municípios que apresentam Rt < 1
dfrtl = dfnna.query("rt_10days_ago_avg < 1")\
            .reset_index(drop=True)\
            .sort_values(['rt_10days_ago_avg'])

dfrtl

,city_id,city_name,state_id,state_name,population,health_system_region,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_ventilators,author_number_ventilators,number_ventilators,confirmed_cases,active_cases,deaths,notification_rate,state_notification_rate,last_updated_subnotification,subnotification_place_type,subnotification_rate,subnotification_rank,subnotification_classification,last_updated_rt,rt_10days_ago_low,rt_10days_ago_high,rt_10days_ago_most_likely,rt_17days_ago_low,rt_17days_ago_high,rt_17days_ago_most_likely,rt_classification,rt_10days_ago_avg,rt_17days_ago_avg,rt_ratio_week_avg,rt_growth,last_updated_inloco,inloco_today_7days_avg,inloco_last_week_7days_avg,inloco_ratio_week_avgs,inloco_growth,dday_beds_best,dday_beds_worst,dday_classification,dday_beds_best_months,overall_alert,data_last_refreshed
1,4213500,Porto Belo,SC,Santa Catarina,21388,Balneário Camboriú,2020-04-01,DataSUS,0,2020-04-01,DataSUS,0.0,27.0,34.0,1.0,0.502857,1.00000,2020-06-09 16:55:47,city,0.497143,35.0,bom,2020-06-09 17:34:37,0.51,1.45,1.00,0.38,1.33,0.85,insatisfatório,0.878571,0.852857,1.030151,estabilizando,2020-06-09 16:57:04,0.390768,0.399671,0.977723,estabilizando,1.0,1.0,ruim,1.0,alto,2020-06-09 17:35:32
0,4314902,Porto Alegre,RS,Rio Grande do Sul,1483771,CRS 01 Porto Alegre,2020-04-01,DataSUS,4912,2020-04-01,DataSUS,1374.0,1139.0,1154.0,44.0,0.464544,0.83431,2020-06-09 16:55:47,city,0.535456,46.0,insatisfatório,2020-06-09 17:34:37,0.84,1.19,1.02,0.86,1.22,1.04,insatisfatório,0.991429,1.074286,0.922872,estabilizando,2020-06-09 16:57:04,0.425936,0.434418,0.980475,estabilizando,91.0,91.0,bom,3.0,medio,2020-06-09 17:35:32


Em apenas {{dfrtl.shape[0]}} município(s) o Rt está abaixo de 0

## Load historical data

In [16]:
# Abrir arquivo com a série histórica da doença
dfh = pd.read_csv('http://localhost:80/br/cities/cases/full')

In [17]:
dfh

,city,city_id,last_updated,epidemiological_week,estimated_population_2019,is_last,is_repeated,confirmed_cases,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,deaths,order_for_place,place_type,state,daily_cases,new_deaths,infectious_period_cases,state_notification_rate,notification_rate,active_cases,data_last_refreshed
0,São Paulo,3550308,2020-02-25,9,12252023.0,False,False,1,0.00816,2020-02-25,0.0000,0,1,city,SP,1.0,0,1.0,NaN,NaN,1.0,2020-06-07 00:55:41
1,São Paulo,3550308,2020-02-26,9,12252023.0,False,False,1,0.00816,2020-02-26,0.0000,0,2,city,SP,0.0,0,1.0,NaN,NaN,1.0,2020-06-07 00:55:41
2,São Paulo,3550308,2020-02-27,9,12252023.0,False,False,1,0.00816,2020-02-27,0.0000,0,3,city,SP,0.0,0,1.0,NaN,NaN,1.0,2020-06-07 00:55:41
3,São Paulo,3550308,2020-02-28,9,12252023.0,False,False,2,0.01632,2020-02-28,0.0000,0,4,city,SP,1.0,0,2.0,NaN,NaN,2.0,2020-06-07 00:55:41
4,São Paulo,3550308,2020-02-29,9,12252023.0,False,False,2,0.01632,2020-02-29,0.0000,0,5,city,SP,0.0,0,2.0,NaN,NaN,2.0,2020-06-07 00:55:41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168459,Tocantínia,1721109,2020-06-06,23,7545.0,False,True,2,26.50762,2020-06-05,0.0000,0,19,city,TO,0.0,0,1.0,1.0,1.0,1.0,2020-06-07 00:55:41
168460,Tupirama,1721257,2020-06-06,23,1891.0,False,True,0,0.00000,2020-06-05,0.0000,0,13,city,TO,0.0,0,0.0,1.0,1.0,0.0,2020-06-07 00:55:41
168461,Tupiratins,1721307,2020-06-06,23,2671.0,False,True,0,0.00000,2020-06-05,0.0000,0,13,city,TO,0.0,0,0.0,1.0,1.0,0.0,2020-06-07 00:55:41
168462,Wanderlândia,1722081,2020-06-06,23,11683.0,False,True,23,196.86724,2020-06-05,0.0000,0,40,city,TO,0.0,0,10.0,1.0,1.0,10.0,2020-06-07 00:55:41


In [18]:
# Mostrar o dia mais recente da série de dados
dfh['last_updated'].max()

'2020-06-06'

In [19]:
#Cálculo da média móvel das novas mortes
def get_rolling_amount(grp):
    return grp.rolling(5, min_periods=1, on='last_updated')['new_deaths'].mean()


df_cities_deaths = dfh[~dfh['deaths'].isnull()][['state','city','city_id', 'last_updated', 'deaths','new_deaths']]\
                                                .groupby(['state','city','city_id', 'last_updated'])[['deaths','new_deaths']]\
                                                .sum()\
                                                .reset_index()

df_cities_deaths['rolling_deaths_new'] = df_cities_deaths.groupby('city', 
                                                    as_index=False, 
                                                    group_keys=False)\
                                           .apply(get_rolling_amount)

In [20]:
#Nome das cidades por ordem (Pode variar de acordo com o que quisermos listar)
##(e.g. Variação do Rt ou valor absoluto do Rt)
cities_names_all = list(dfnfilt['city_name'])

cities_names = list(dfnfilt.sort_values('rt_10days_ago_avg', ascending = False)\
                                             ['city_name'].iloc[:10])

#ID da cidades por ordem (Pode variar de acordo com o que quisermos listar)
##(e.g. Variação do Rt ou valor absoluto do Rt)
cities_id_all = list(dfnfilt['city_id'])

cities_id = list(dfnfilt.sort_values('rt_10days_ago_avg', ascending = False)\
                                             ['city_id'].iloc[:10])

#Selecionar apesar os municípios listados em city_id
t = df_cities_deaths.query(f'city_id =={cities_id}')


In [21]:
first_death = [dfh.query(f'city_id =={j}')\
    .query(f'deaths !=0')['last_updated'].min() for j in cities_id_all 
]

first_case = [dfh.query(f'city_id =={j}')\
    .query(f'confirmed_cases != 0')['last_updated'].min() for j in cities_id_all 
]

In [22]:
#Titulos dos gráficos
titles = [
    'Distribuição de mortes nas 10 cidades com o menor ritmo de contágio',
    'Distribuição de mortes nas 10 cidades com o maior ritmo de contágio',
    'Distribuição de mortes nas 10 cidades com o maior diminuição ritmo de contágio',
    'Distribuição de mortes nas 10 cidades com o maior aumento ritmo de contágio'
]

In [23]:
v = 0

In [24]:
place_type='city'
col_time= 'last_updated'
#min_deaths=10
title="Mapa de calor: Média móvel de novas mortes nos últimos 5 dias"
colors='temps'

In [25]:
#Formatar o Dataset para a plotagem
df_heatmap = t.reset_index()\
                              .pivot(index='city', 
                                      columns='last_updated', 
                                      values='rolling_deaths_new')\
                               .fillna(0)\
                               .apply(lambda x: x/x.max(), axis=1)\
                               .dropna(how='all')

# remove dias com todos os valores iguais a 0
df_heatmap = df_heatmap.loc[:, (df_heatmap != 0).any(axis=0)]
print(len(df_heatmap))

city_total_deaths = [t[t['city'] ==x]['deaths'].max() for x in df_heatmap.loc[cities_names].index]



10


### Visualização da distribuições das mortes nos municípios selecionados

In [26]:
df_heatmap.loc[cities_names]

last_updated,2020-04-02,2020-04-07,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06
city,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Barcarena,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.021277,0.021277,0.021277,0.021277,0.000000,0.000000,0.000000,0.042553,0.042553,0.042553,0.042553,0.042553,0.000000,0.042553,0.042553,0.042553,0.085106,0.106383,0.063830,0.063830,0.063830,0.021277,0.000000,0.063830,0.063830,0.787234,0.808511,0.978723,1.000000,1.000000,0.297872,0.319149,0.148936,0.063830,0.042553,0.063830,0.021277,0.021277,0.063830,0.063830,0.021277,0.021277,0.021277
Paragominas,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.072464,0.108696,0.086957,0.130435,0.130435,0.086957,0.043478,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.260870,0.478261,0.478261,0.695652,0.695652,0.521739,0.956522,0.956522,0.782609,1.000000,0.913043,0.260870,0.304348,0.652174,0.608696,0.608696,0.608696,0.565217
Curuçá,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.176471,0.176471,0.176471,0.176471,0.176471,0.000000,0.000000,0.411765,0.411765,0.411765,0.411765,0.470588,0.058824,0.058824,0.058824,0.058824,0.705882,0.705882,0.882353,0.941176,1.000000,0.411765,0.411765,0.470588,0.411765,0.411765,0.352941,0.352941,0.117647,0.117647,0.058824,0.000000,0.000000,0.000000
Tucuruí,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.136364,0.136364,0.136364,0.090909,0.227273,0.136364,0.136364,0.136364,0.136364,0.045455,0.045455,0.045455,0.045455,0.500000,0.681818,0.681818,1.000000,1.000000,0.727273,0.500000,0.500000,0.454545,0.500000,0.318182,0.590909,0.590909,0.318182,0.272727,0.272727,0.000000,0.136364,0.136364,0.136364,0.136364
Abaetetuba,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.043478,0.130435,0.260870,0.304348,0.347826,0.347826,0.260870,0.173913,0.130435,0.130435,0.130435,0.260870,0.304348,0.695652,0.913043,0.913043,0.869565,1.000000,0.869565,0.652174,0.782609,0.739130,0.521739,0.260870,0.478261,0.478261,0.434783,0.434783,0.434783
Benevides,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.045455,0.045455,0.045455,0.000000,0.090909,0.181818,0.181818,0.181818,0.181818,0.090909,0.000000,0.000000,0.045455,0.090909,0.136364,0.136364,0.181818,0.136364,0.090909,0.045455,0.090909,0.045455,0.045455,0.045455,0.090909,0.136364,0.136364,0.181818,0.181818,0.181818,0.090909,0.090909,0.409091,0.409091,1.000000,1.000000,1.000000,0.727273,0.772727,0.136364,0.136364,0.136364
São Vicente,0.023256,-0.023256,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.023256,0.000000,0.023256,0.023256,0.023256,0.000000,0.023256,0.000000,0.000000,0.023256,0

In [27]:
# Plotar o mapa de calor com total de mortes por município
fig = heatmap(df_heatmap.loc[cities_names].iloc[::-1], city_total_deaths[::-1], title, colors)
fig.show()

In [28]:
import plotly.io as pio

In [29]:
#Salvar imagem
#pio.write_image(fig,f'../data/figures/hm0{v}.jpeg',format='jpeg',width=1920, height=1080)
#v+=1

In [30]:
dfnfilt['first_case'] = first_case
dfnfilt['first_death'] = first_death

In [31]:
#DataFrame com os dados ordenados
##Alterando o nome da coluna, altera a ordenação dos dados, as 10 primeiras linhas serão
## as 10 menores observações, as 10 ultimas, serão as 10 maiores
### e.g. Municipios com o maior Rt, municípios com o menor Rt ...
dfexib = dfnfilt.sort_values('rt_10days_ago_avg', ascending = False) #Alterar essa linha

In [32]:
h = dfexib[['city_name','state_id','first_death','deaths','deaths_100khab',
 'first_case','confirmed_cases', 'cases_100khab', 'subnotification_rate',
        'inloco_last_week_7days_avg','inloco_today_7days_avg',
        'rt_17days_ago_avg','rt_10days_ago_avg']]
h.to_csv("..data/output/dados_g1.csv", index=False)

In [33]:
#Exibição dos dados mostrando algumas variáveis relevantes
## No exemplo, municípios com os 10 menores Rt
h.iloc[:10]

,city_name,state_id,first_death,deaths,deaths_100khab,first_case,confirmed_cases,cases_100khab,subnotification_rate,inloco_last_week_7days_avg,inloco_today_7days_avg,rt_17days_ago_avg,rt_10days_ago_avg
117,Barcarena,PA,2020-04-24,68.0,54.539621,2020-04-01,983.0,788.418351,0.733238,0.410981,0.373488,2.152857,2.362857
123,Paragominas,PA,2020-04-26,64.0,56.564585,2020-04-24,940.0,830.792346,0.735089,0.438382,0.409911,1.374286,2.161429
72,Curuçá,PA,2020-05-04,40.0,99.835272,2020-04-22,475.0,1185.543853,0.784588,0.454609,0.408861,2.167143,2.067143
27,Tucuruí,PA,2020-05-02,59.0,51.909660,2020-03-31,696.0,612.358018,0.765782,0.470149,0.429766,2.318571,2.048571
122,Abaetetuba,PA,2020-05-08,68.0,43.120395,2020-03-31,1735.0,1100.204188,0.472154,0.484150,0.447712,1.634286,1.972857
121,Benevides,PA,2020-04-24,43.0,68.540096,2020-04-03,1034.0,1648.150214,0.545170,0.411787,0.375134,1.622857,1.887143
119,São Vicente,SP,2020-04-02,91.0,24.877118,2020-03-31,1352.0,369.602896,0.719370,0.420543,0.416872,1.642857,1.837143
120,Campina Grande,PB,2020-04-16,44.0,10.738753,2020-03-27,2650.0,646.765805,0.000000,0.459431,0.431114,1.592857,1.834286
1,Marituba,PA,2020-04-16,48.0,36.496073,2020-03-31,497.0,377.886421,0.793501,0.406587,0.381179,2.304286,1.808571
109,Magé,RJ,2020-04-13,98.0,39.988412,2020-04-02,918.0,374.585324,0.835686,0.439057,0.427219,1.721429,1.775714


In [34]:
print("Max: %f" % dfnfilt['rt_10days_ago_avg'].sort_values(ascending = False)[:10].max())
print("Min: %f" % dfnfilt['rt_10days_ago_avg'].sort_values(ascending = False)[:10].min())



Max: 2.362857
Min: 1.775714
